## 一、載入套件與設定路徑

In [15]:
import pandas as pd
import numpy as np
import os
import warnings

# 忽略 pandas 的警告訊息
warnings.filterwarnings('ignore')

# --- 📂 Step 1: 設定檔案路徑 ---
# 根據您的指示，直接設定相對路徑
data_dir = "../DataSet/"
output_dir = "./" # 輸出到當前目錄

# 確保輸出目錄存在
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(f"📂 資料讀取路徑設定為: {data_dir}")

📂 資料讀取路徑設定為: ../DataSet/


## 二、讀取原始資料

In [16]:
# --- 📥 Step 2: 讀取資料 ---
print("⏳ 正在讀取資料...")

try:
    # 1. 讀取交易紀錄
    txn_path = os.path.join(data_dir, 'acct_transaction.csv')
    df_txn = pd.read_csv(txn_path, dtype={
        'from_acct': str, 'to_acct': str, 
        'from_acct_type': str, 'to_acct_type': str,
        'txn_amt': float
    })

    # 2. 讀取警示帳戶 (標籤資料)
    alert_path = os.path.join(data_dir, 'acct_alert.csv')
    df_alert = pd.read_csv(alert_path, dtype={'acct': str})

    # 3. 讀取待預測帳戶 (提交目標)
    # 加入欄位名稱檢查機制，確保能正確讀取 ID
    predict_path = os.path.join(data_dir, 'acct_predict.csv')
    # 先讀取 header 確認欄位
    df_predict_raw = pd.read_csv(predict_path, nrows=0)
    predict_cols = df_predict_raw.columns.tolist()
    
    # 判斷 ID 欄位名稱
    if 'acct_test' in predict_cols:
        pid = 'acct_test'
    elif 'acct' in predict_cols:
        pid = 'acct'
    else:
        pid = predict_cols[0] # 預設第一個欄位
    
    df_predict = pd.read_csv(predict_path, dtype={pid: str})
    if pid != 'acct_test':
        df_predict = df_predict.rename(columns={pid: 'acct_test'})

    print(f"✅ 資料讀取完成！")
    print(f"   - 交易紀錄: {df_txn.shape[0]} 筆")
    print(f"   - 警示帳戶: {df_alert.shape[0]} 筆")
    print(f"   - 待預測帳戶: {df_predict.shape[0]} 筆")

except FileNotFoundError as e:
    print(f"❌ 錯誤：找不到檔案，請確認路徑是否正確。\n詳細錯誤: {e}")
    # 為了讓下面的 cell 不報錯，定義空的 DataFrame (僅在出錯時)
    df_txn = pd.DataFrame()
    df_alert = pd.DataFrame()
    df_predict = pd.DataFrame()

⏳ 正在讀取資料...
✅ 資料讀取完成！
   - 交易紀錄: 4435890 筆
   - 警示帳戶: 1004 筆
   - 待預測帳戶: 4780 筆


## 三、特徵工程

In [17]:
# --- 🛠️ Step 3: 特徵工程 (Feature Engineering) ---
if not df_txn.empty:
    print("⚙️ 開始計算 V4 官方特徵...")

    # 1. 處理玉山帳戶標記
    df_txn['is_esun_from'] = (df_txn['from_acct_type'] == '01').astype(int)
    df_txn['is_esun_to'] = (df_txn['to_acct_type'] == '01').astype(int)

    # --- 付款方特徵 ---
    print("   計算付款方特徵...")
    debit_features = df_txn.groupby('from_acct').agg(
        total_send_amt=('txn_amt', 'sum'),
        max_send_amt=('txn_amt', 'max'),
        min_send_amt=('txn_amt', 'min'),
        avg_send_amt=('txn_amt', 'mean'),
        count_send_txns=('txn_amt', 'count'),
        is_esun_from=('is_esun_from', 'max')
    ).reset_index().rename(columns={'from_acct': 'acct'})

    # --- 收款方特徵 ---
    print("   計算收款方特徵...")
    credit_features = df_txn.groupby('to_acct').agg(
        total_recv_amt=('txn_amt', 'sum'),
        max_recv_amt=('txn_amt', 'max'),
        min_recv_amt=('txn_amt', 'min'),
        avg_recv_amt=('txn_amt', 'mean'),
        count_recv_txns=('txn_amt', 'count'),
        is_esun_to=('is_esun_to', 'max')
    ).reset_index().rename(columns={'to_acct': 'acct'})

    # --- 合併 ---
    print("   合併特徵...")
    df_features = pd.merge(debit_features, credit_features, on='acct', how='outer')

    # 填補 NaN
    fillna_cols = [
        'total_send_amt', 'max_send_amt', 'min_send_amt', 'avg_send_amt', 'count_send_txns', 'is_esun_from',
        'total_recv_amt', 'max_recv_amt', 'min_recv_amt', 'avg_recv_amt', 'count_recv_txns', 'is_esun_to'
    ]
    df_features[fillna_cols] = df_features[fillna_cols].fillna(0)
    df_features['is_esun'] = df_features[['is_esun_from', 'is_esun_to']].max(axis=1)

    print(f"✅ 特徵計算完成！共 {df_features.shape[0]} 個帳戶。")

⚙️ 開始計算 V4 官方特徵...
   計算付款方特徵...
   計算收款方特徵...
   合併特徵...
✅ 特徵計算完成！共 1800106 個帳戶。


## 製作訓練集與測試集

In [18]:
    # --- ✂️ Step 4: 資料分割 (Data Split) ---
    print("📦 分割訓練集與測試集...")

    # 標記標籤
    df_features_labeled = pd.merge(df_features, df_alert, on='acct', how='left', indicator=True)
    df_features_labeled['label'] = (df_features_labeled['_merge'] == 'both').astype(int)
    df_features_labeled = df_features_labeled.drop(columns=['_merge'])

    predict_acct_set = set(df_predict['acct_test'])

    # 測試集 (Test)
    df_test = df_features_labeled[df_features_labeled['acct'].isin(predict_acct_set)].copy()
    
    # 補齊遺漏帳戶
    missing = predict_acct_set - set(df_test['acct'])
    if missing:
        print(f"⚠️ 補齊 {len(missing)} 個無交易紀錄帳戶。")
        miss_df = pd.DataFrame({'acct': list(missing)})
        for c in df_test.columns:
            if c != 'acct': miss_df[c] = 0
        df_test = pd.concat([df_test, miss_df], ignore_index=True)
    
    # 確保順序並移除不必要欄位
    df_test = pd.merge(df_predict, df_test, left_on='acct_test', right_on='acct', how='left')
    # 修改後 (穩健寫法)
    # errors='ignore' 代表如果欄位不存在，不要報錯，繼續執行
    df_test = df_test.drop(columns=['acct_test', 'label'], errors='ignore')

    
    # 重新命名 acct 為 acct_test 以統一標籤
    df_test = df_test.rename(columns={'acct': 'acct_test'})

    # 訓練集 (Train)
    df_train = df_features_labeled[~df_features_labeled['acct'].isin(predict_acct_set)].copy()

    print(f"✅ 分割完成！ Train: {df_train.shape}, Test: {df_test.shape}")

📦 分割訓練集與測試集...
✅ 分割完成！ Train: (1795326, 16), Test: (4780, 17)


## 儲存處理後的檔案

In [19]:
# --- 💾 Step 5: 儲存檔案 ---
print("💾 儲存檔案...")
df_train.to_csv(os.path.join(output_dir, 'train_features_v4.csv'), index=False)
df_test.to_csv(os.path.join(output_dir, 'test_features_v4.csv'), index=False)
print("🎉 所有步驟執行完畢！")

💾 儲存檔案...
🎉 所有步驟執行完畢！
